In [1]:
from preprocess import *
import pandas as pd
import numpy as np

In [2]:
fraud_data_raw = pd.read_csv('/Users/JasonLebov_1/Desktop/CS 334/Final Project/Final Project Data/fraud test.csv')

In [3]:
# Step 1: Dropping Features
features_drop = ['Unnamed: 0', 'street', 'city', 'state', 'zip', 'unix_time', 'trans_num']
drop_features_df = drop_features(fraud_data_raw, features_drop)

In [4]:
# Step *: Removing 'fraud_' prefix from merchant
removed_prefix_df = remove_prefix(drop_features_df, 'merchant')

In [5]:
# Step 2: Feature Extraction --> 'trans_date_trans_time'
     # Converting to DateTime Object
col_name_1 = 'trans_date_trans_time'
col_format_1 = '%d/%m/%Y %H:%M'
fixed_dt_1 = convert_dt(removed_prefix_df, col_name_1, col_format_1)

    # Extracting Values
fixed_dt_1['transaction_month'] = fixed_dt_1['trans_date_trans_time'].dt.month
fixed_dt_1['transaction_day'] = fixed_dt_1['trans_date_trans_time'].dt.day
fixed_dt_1['transaction_year'] = fixed_dt_1['trans_date_trans_time'].dt.year
fixed_dt_1['transaction_time'] = fixed_dt_1['trans_date_trans_time'].dt.strftime('%H:%M')

    # Converting Time to Categorical Ranges
fixed_dt_1 = convert_time_ranges(fixed_dt_1, 'transaction_time')

    # Dropping Original 'trans_date_trans_time' 
fixed_dt_1 = drop_features(fixed_dt_1, ['trans_date_trans_time'])

In [7]:
# Step 2 [cont]: Feature Extraction --> Year Born from 'dob'
     # Converting to DateTime Object
col_name_2 = 'dob'
col_format_2 = '%d/%m/%Y'
fixed_dt_2 = convert_dt(fixed_dt_1, col_name_2, col_format_2)

    # Extracting Values
fixed_dt_2['year_born'] = fixed_dt_2['dob'].dt.year

    # Dropping Original 'dob' 
final_extraction_df = drop_features(fixed_dt_2, ['dob'])

In [9]:
# Step 3: One Hot Encode
cols_encode = ['category', 'gender', 'transaction_time']
one_hot_df = one_hot_encode(final_extraction_df, cols_encode)

In [13]:
one_hot_df.head()

,cc_num,merchant,amt,first,last,lat,long,city_pop,job,merch_lat,...,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel,gender_F,gender_M,transaction_time_morning,transaction_time_afternoon,transaction_time_evening
0,2.291160e+15,Kirlin and Sons,2.86,Jeff,Elliott,33.9659,-80.9355,333497,Mechanical engineer,33.986391,...,0,1,0,0,0,0,1,0,1,0
1,3.573030e+15,Sporer-Keebler,29.84,Joanne,Williams,40.3207,-110.4360,302,"Sales professional, IT",39.450498,...,0,1,0,0,0,1,0,0,1,0
2,3.598220e+15,"Swaniawski, Nitzsche and Welch",41.28,Ashley,Lopez,40.6729,-73.5365,34496,"Librarian, public",40.495810,...,0,0,0,0,0,1,0,0,1,0
3,3.591920e+15,Haley Group,60.05,Brian,Williams,28.5697,-80.8191,54767,Set designer,28.812398,...,1,0,0,0,0,0,1,0,1,0
4,3.526830e+15,Johnston-Casper,3.19,Nathan,Massey,44.2529,-85.0170,1126,Furniture designer,44.959148,...,0,0,0,0,1,0,1,0,1,0


In [15]:
one_hot_df.dtypes

cc_num                        float64
merchant                       object
amt                           float64
first                          object
last                           object
lat                           float64
long                          float64
city_pop                        int64
job                            object
merch_lat                     float64
merch_long                    float64
is_fraud                        int64
transaction_month               int32
transaction_day                 int32
transaction_year                int32
year_born                       int32
category_entertainment          int64
category_food_dining            int64
category_gas_transport          int64
category_grocery_net            int64
category_grocery_pos            int64
category_health_fitness         int64
category_home                   int64
category_kids_pets              int64
category_misc_net               int64
category_misc_pos               int64
category_per

In [23]:
# Step 4: Vectorize Categorical Variables (GloVe)

# Names --> First & Last

    # Convert All Values to Lower Case
one_hot_df['first'] = one_hot_df['first'].str.lower()
one_hot_df['last'] = one_hot_df['last'].str.lower()

first_names = list(one_hot_df['first'])


In [25]:
first_names = list(one_hot_df['first'])